In [1]:
import tensorflow as tf
import gym
from tensorforce import Agent, Environment, Runner

from mountain_car_wrappers import RewardEnergy

In [3]:
with gym.make('MountainCarContinuous-v0') as env:
    env = RewardEnergy(env)
    environment = Environment.create(
        environment=env, max_episode_timesteps=999
    )

    network = [
        dict(type='dense', size=32, activation='relu'),
        dict(type='dense', size=32, activation='relu')
    ]

    epsilon = dict(
        type='exponential', unit='episodes', num_steps=100,
        initial_value=1.0, decay_rate=.05, dtype=tf.float32
    )

    mcc_agent = Agent.create(
        agent='dpg', environment=environment, memory=50000, batch_size=256, network=network,
        use_beta_distribution=True, update_frequency=4, start_updating=2000,
        learning_rate=1e-3, discount=0.99, exploration=epsilon,
        critic=network
    )

    runner = Runner(
        agent=mcc_agent, environment=environment, max_episode_timesteps=999
    )

    runner.run(num_episodes=100, save_best_agent='./mcc_tf/')
    runner.run(num_episodes=100, evaluation=True)
    runner.close()

Episodes:   0%|          | 0/100 [00:00, return=0.00, ts/ep=0, sec/ep=0.00, ms/ts=0.0, agent=0.0%]

AssertionError: 

In [3]:
from gym.wrappers import Monitor
with Monitor(gym.make('MountainCarContinuous-v0'), './mcc_dpg_demo/', force=True) as env:
    done = False
    obs = env.reset()
    total_reward = 0
    internals = mcc_agent.initial_internals()
    while not done:
        env.render()
        action = mcc_agent.act(states=obs, internals=internals, independent=True, deterministic=True)
        obs, rwd, done, _ = env.step(action[0])
        total_reward += rwd
        
    print(f'Total Reward: {total_reward}')

2022-07-06 16:33:12.286 python[89475:578950] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fcf4bea8c80>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-07-06 16:33:12.288 python[89475:578950] Warning: Expected min height of view: (<NSButton: 0x7fcf5073b9e0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-07-06 16:33:12.290 python[89475:578950] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fcf50b3cb90>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-07-06 16:33:12.292 python[89475:578950] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fcf4bec9730>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


Total Reward: -99.89959509413625
